This notebook contains code used for StreamLit input. This is a simplified model so that it only includes fields that an investor would have easy access to, but would still be accurate enough to use.

# Import Data

In [4]:
#Install necessary packages
import os #needed to check directory
import pandas as pd #import pandas library to convert data to dataframe for easy analysis
import numpy as np #import numpy library for EDA
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # more data visualization and statistics
import sklearn #machine learning
import plotly.express as px #plotting
from ast import literal_eval #used to convert string to list
from sklearn import preprocessing #for label encoding and normalizing
from sklearn.preprocessing import MinMaxScaler #to normalize data
from sklearn.preprocessing import PolynomialFeatures #for polynomial regression
from sklearn import metrics #simple MAE, MSE, RMSE calculation packages
from scipy.stats import gaussian_kde #dispay density scatter chart
from sklearn.model_selection import train_test_split #model splitting into training and testing data
from sklearn.linear_model import LinearRegression #package for linear regression model
!pip install lazypredict #run data against many models
import nltk #natural language processing tool kit
nltk.download('punkt') #tokenize words in dataframe
nltk.download('stopwords') #identifying stopwords as part of nlp
from sklearn import metrics #use sklearn built-in calculations for scoring success metrics
from scipy.spatial.distance import cdist #calculates distance between many coordinates at once
from lazypredict import Supervised #laxypredict for supervised learning problem
from lazypredict.Supervised import LazyRegressor #laxypredict for regression problem
import lightgbm as ltb #LigthGBM Model for ML
#import data from google drivefrom sklearn.preprocessing import PolynomialFeatures #Polynomial fitting model
import plotly.graph_objects as go #allows 2 sets of data on one set of axes plotly

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Access data from google drive & reformat it for import

url = 'https://drive.google.com/file/d/1YU91bN5I9mIj75if3T12dVWbxNco95Wd/view?usp=drive_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2] #reformat

#define a dataframe called abnb0 containing all listing data. Specify the datatypes.
abnb0 = pd.read_csv(path, dtype={"listing_url": "string","scrape_id": "float","last_scraped": "string","name": "string",
                    "description": "string","neighborhood_overview": "string","picture_url": "string",
                    "host_id": "float","host_url": "string","host_name": "string","host_since": "string",
                    "host_location": "string","host_about": "string","host_response_time": "string",
                    "host_response_rate": "string","host_acceptance_rate": "string","host_is_superhost": "string",
                    "host_thumbnail_url": "string","host_picture_url": "string","host_neighbourhood": "string",
                    "host_listings_count": "float","host_total_listings_count": "float","host_verifications": "string",
                    "host_has_profile_pic": "string","host_identity_verified": "string","neighbourhood": "string",
                    "neighbourhood_cleansed": "string","neighbourhood_group_cleansed": "string","latitude": "float",
                    "longitude": "float","property_type": "string","room_type": "string","accommodates": "float",
                    "bathrooms": "float","bathrooms_text": "string","bedrooms": "float","beds": "float",
                    "amenities": "string","price": "string","minimum_nights": "float","maximum_nights": "float",
                    "minimum_minimum_nights": "float","maximum_minimum_nights": "float","minimum_maximum_nights": "float",
                    "maximum_maximum_nights": "float","minimum_nights_avg_ntm": "float","maximum_nights_avg_ntm": "float",
                    "calendar_updated": "string","has_availability": "string","availability_30": "int64",
                    "availability_60": "float","availability_90": "float","availability_365": "float",
                    "calendar_last_scraped": "string","number_of_reviews": "float","number_of_reviews_ltm": "float",
                    "number_of_reviews_l30d": "float","first_review": "string","last_review": "string",
                    "review_scores_rating": "float","review_scores_accuracy": "float",
                    "review_scores_cleanliness": "float","review_scores_checkin": "float",
                    "review_scores_communication": "float","review_scores_location": "float",
                    "review_scores_value": "float","license": "string","instant_bookable": "string",
                    "calculated_host_listings_count": "float","calculated_host_listings_count_entire_homes": "float",
                    "calculated_host_listings_count_private_rooms": "float",
                    "calculated_host_listings_count_shared_rooms": "float","reviews_per_month": "float"})


# StreamLit

In [22]:
abnbSL=abnb0.copy()

In [24]:
def reformClean2(df):
  #To fix the price column, use a regex to replace the '$' and convert to float
  df['price'] = df['price'].str.replace(',', '', regex=True).str.replace('$', '', regex=True).astype(float)

  #bathrooms_text has mixed information containing the number of bathrooms and the type of bathroom
  df['bathrooms_text'] = df['bathrooms_text'].replace(['Shared half-bath', 'Private half-bath', 'Half-bath'],
                                              ['0.5 shared', '0.5 private', '0.5 private'])
  # split the bathrooms_text column into 2 new columns where the delimiter is the *first* space in the text
  df[['new_bathroom','new_bathroom_type']] = df['bathrooms_text'].str.split(' ',n=1, expand=True)
  #convert this column into float. Coerce removes the error with converting NAs.
  df['new_bathroom'] = df['new_bathroom'].apply(pd.to_numeric, errors='coerce')

  df['new_bathroom_type'].replace({'bath':'private'},inplace=True)
  df['new_bathroom_type'].replace({'baths':'private'},inplace=True)
  df['new_bathroom_type'].replace({'private bath':'private'},inplace=True)
  df['new_bathroom_type']=df['new_bathroom_type'].replace('shared bath','shared')
  df['new_bathroom_type']=df['new_bathroom_type'].replace('shared baths','shared')

    #rename neighborhood column to more logical title
  df.rename(columns={'neighbourhood_group_cleansed': 'borough'}, inplace=True)

  #delete the rows that have 0 for rating as these are auto generated when a host cancels last minute
  index_names1 = df[ df['review_scores_rating'] == 0 ].index
  df.drop(index_names1, inplace = True)

  #delete the bnbs that have less than 3 total reviews
  index_names2 = df[ df['number_of_reviews'] < 3].index
  df.drop(index_names2, inplace = True)


  #fill nulls with appropriate values
  df.bedrooms=df.bedrooms.fillna(0) #likely a studio
  df.beds=df.beds.fillna(1) #likely has one bed
  df.review_scores_rating=df.review_scores_rating.fillna(df['review_scores_rating'].mean())
  df.review_scores_accuracy=df.review_scores_accuracy.fillna(df['review_scores_accuracy'].mean())
  df.review_scores_cleanliness=df.review_scores_cleanliness.fillna(df['review_scores_cleanliness'].mean())
  df.review_scores_checkin=df.review_scores_checkin.fillna(df['review_scores_checkin'].mean())
  df.review_scores_communication=df.review_scores_communication.fillna(df['review_scores_communication'].mean())
  df.review_scores_location=df.review_scores_location.fillna(df['review_scores_location'].mean())
  df.review_scores_value=df.review_scores_value.fillna(df['review_scores_value'].mean())
  df.reviews_per_month=df.reviews_per_month.fillna(df['reviews_per_month'].mean())
  df.new_bathroom=df.new_bathroom.fillna(1) #likely one bathroom
  df.new_bathroom_type=df.new_bathroom_type.fillna('private') #likely private
  df.host_is_superhost=df.host_is_superhost.fillna('f') #likely not a superhost

  #delete the rows that have $0 for price as these will not help the models
  index_names2 = df[df['price'] == 0].index
  df.drop(index_names2, inplace = True)

  #remove outliers by using interquartile values for price
  Q1 = df['price'].quantile(0.25)
  Q3 = df['price'].quantile(0.75)
  IQR = Q3 - Q1

  index_names = df[(df['price'] > Q3 + 1.5 * IQR)].index
  df.drop(index_names , inplace=True)

  #reset index
  df.reset_index(inplace=True)
  #rename neighborhood column to more logical title
  df.rename(columns={'neighbourhood_group_cleansed': 'borough'}, inplace=True)

  #exclude staten island
  df1 = df[(df['borough'] == "Staten Island")].index
  df.drop(df1, inplace = True)

  #clean out bnbs incorrectly labelled Manhattan by location
  index4 = df[ (df['latitude'] >= 40.71101) & (df['longitude'] <= -74.0182) ].index
  df.drop(index4 , inplace=True)

  index5 = df[ (df['latitude'] >= 40.76796) & (df['longitude'] <= -73.99703) ].index
  df.drop(index5 , inplace=True)

  index6 = df[ (df['latitude'] >= 40.79505) & (df['longitude'] <= -73.98447) ].index
  df.drop(index6 , inplace=True)

  index7 = df[ (df['latitude'] >= 40.8263) & (df['longitude'] <= -73.96606) ].index
  df.drop(index7 , inplace=True)

#remove columns that do not contain pertinent information to help predict abnb price
  df.drop(columns=['listing_url', 'scrape_id', 'last_scraped', 'name', 'description','id',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'latitude', 'longitude',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'bathrooms',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews', 'bathrooms_text',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_accuracy','review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'], inplace=True)

In [25]:
reformClean2(abnbSL)

In [26]:
abnbSL.head()

,index,host_is_superhost,neighbourhood_cleansed,borough,property_type,room_type,accommodates,bedrooms,beds,amenities,price,review_scores_rating,new_bathroom,new_bathroom_type
0,0,f,Midtown,Manhattan,Entire rental unit,Entire home/apt,1.00,0.00,1.00,"[""Essentials"", ""Bathtub"", ""Extra pillows and b...",225.00,4.70,1.00,private
1,1,f,Bedford-Stuyvesant,Brooklyn,Private room in rental unit,Private room,2.00,1.00,1.00,"[""Heating"", ""Long term stays allowed"", ""Kitche...",60.00,4.52,1.00,private
2,3,f,Midtown,Manhattan,Private room in rental unit,Private room,2.00,1.00,1.00,"[""Conditioner"", ""Essentials"", ""Extra pillows a...",68.00,4.23,1.00,private
3,4,f,Upper West Side,Manhattan,Private room in rental unit,Private room,1.00,1.00,1.00,"[""Carbon monoxide alarm"", ""Heating"", ""Essentia...",75.00,4.91,1.00,shared
4,5,t,Morningside Heights,Manhattan,Entire rental unit,Entire home/apt,2.00,1.00,1.00,"[""Carbon monoxide alarm"", ""Shampoo"", ""Essentia...",135.00,4.68,1.00,private


In [27]:
#Encode
abnbSL = abnbSL.replace({'room_type' : {'Hotel room' : '3', 'Entire home/apt' : '4', 'Private room' : '2', 'Shared room' : '1'}}).astype({'room_type':'int'})
abnbSL = abnbSL.replace({'new_bathroom_type' : {'private' : '1', 'shared' : '0'}}).astype({'new_bathroom_type':'int'})
abnbSL = abnbSL.replace({'host_is_superhost' : {'t' : '1', 'f' : '0'}}).astype({'host_is_superhost':'int'})

categorical_cols = ['borough']
abnbSL = pd.get_dummies(data = abnbSL, columns = categorical_cols).copy() #one-hot encode neighborhood_group_cleansed

label_encoder = preprocessing.LabelEncoder()
abnbSL.neighbourhood_cleansed= label_encoder.fit_transform(abnbSL.neighbourhood_cleansed).copy() #label encode neighborhood cleansed
abnbSL.property_type= label_encoder.fit_transform(abnbSL.property_type).copy() #label encode property_type

In [28]:
abnbSL.shape

(20851, 17)

In [29]:
#correlation_mat = abnb.drop(['index', 'id', 'latitude','longitude'],axis=1).corr()
corr = abnbSL.drop(['index'],axis=1).corr()
corr_pairs = corr.unstack()
sorted_pairs = corr_pairs.sort_values(kind="quicksort")
strong_pairs = sorted_pairs[abs(sorted_pairs) > 0.5]
strong_pairs = strong_pairs[abs(sorted_pairs) < 1.0]
pd.set_option('display.max_rows', None)
print(strong_pairs)

property_type      room_type           -0.95
room_type          property_type       -0.95
property_type      new_bathroom_type   -0.71
new_bathroom_type  property_type       -0.71
borough_Brooklyn   borough_Manhattan   -0.64
borough_Manhattan  borough_Brooklyn    -0.64
price              new_bathroom_type    0.51
new_bathroom_type  price                0.51
price              room_type            0.53
room_type          price                0.53
accommodates       bedrooms             0.61
bedrooms           accommodates         0.61
beds               bedrooms             0.62
bedrooms           beds                 0.62
beds               accommodates         0.75
accommodates       beds                 0.75
new_bathroom_type  room_type            0.78
room_type          new_bathroom_type    0.78
dtype: float64


In [30]:
#remove roomtype to account for multi-collinearity
abnbSL.drop(columns=['property_type','new_bathroom_type','beds', 'neighbourhood_cleansed','borough_Brooklyn'], inplace=True)

In [31]:
def LELR_all(df):
  X = df.drop(['index', 'price','amenities'],axis=1)
  y=df['price']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=90)

  lm = LinearRegression()
  lm.fit(X_train,y_train)

  coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])
  pd.set_option('display.max_rows', None)
  print(coeff_df.sort_values('Coefficient', ascending=False))

  predictions = lm.predict(X_test)

  ytest=y_test.values

  fig = px.scatter(x=ytest,y=predictions,width=1000, height=1000)
  fig.update_layout(yaxis_range=[-100,400])
  fig.update_layout(xaxis_range=[0,400])
  fig.show()

  print("min of y_test =",min(y_test),"max of y_test =", max(y_test))
  print("min of predictions =",min(predictions),"max of predictions =", max(predictions))

  print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [32]:
LELR_all(abnbSL)

                      Coefficient
borough_Manhattan           40.40
room_type                   27.73
bedrooms                    15.86
host_is_superhost           14.50
review_scores_rating        13.77
accommodates                12.14
new_bathroom                 9.04
borough_Queens             -13.91
borough_Bronx              -23.31


min of y_test = 10.0 max of y_test = 380.0
min of predictions = 2.8626751630337424 max of predictions = 385.5249676749201
RMSE: 58.97030631139417


In [33]:
abnbSL['amenities'] = abnbSL['amenities'].str.replace('[', '').str.replace(']', '').str.replace(',', '').str.replace('"', '').str.lower() # amenities is in type 'string'. replace all unwanted characters and convert to lowercase

In [34]:
#using the best frequent amenities, create new columns that identify if word is present (only if the word may contribute to change in price)
abnbSL["Wifi"] = abnbSL["amenities"].map(lambda x: '1' if "wifi" in x else '0').astype(str).astype(int)
abnbSL["TV"] = abnbSL["amenities"].map(lambda x: '1' if "tv" in x else '0').astype(str).astype(int)
abnbSL["Kitchen"] = abnbSL["amenities"].map(lambda x: '1' if "kitchen" in x else '0').astype(str).astype(int)
abnbSL["Washer"] = abnbSL["amenities"].map(lambda x: '1' if "washer" in x else '0').astype(str).astype(int)
abnbSL["Heating"] = abnbSL["amenities"].map(lambda x: '1' if "heating" in x else '0').astype(str).astype(int)
abnbSL["Air conditioning"] = abnbSL["amenities"].map(lambda x: '1' if "air conditioning" in x else '0').astype(str).astype(int)
abnbSL["Fridge"] = abnbSL["amenities"].map(lambda x: '1' if "refrigerator" in x else '0').astype(str).astype(int)
abnbSL["Cooking"] = abnbSL["amenities"].map(lambda x: '1' if "cooking" in x else '0').astype(str).astype(int)
abnbSL["Essentials"] = abnbSL["amenities"].map(lambda x: '1' if "essentials" in x else '0').astype(str).astype(int)
abnbSL["Hot tub"] = abnbSL["amenities"].map(lambda x: '1' if "hot tub" in x else '0').astype(str).astype(int)
abnbSL.drop(['amenities'], axis=1, inplace=True) #remove the original amenities column

In [35]:
def LELR_all(df):
  X = df.drop(['index', 'price'],axis=1)
  y=df['price']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=90)

  lm = LinearRegression()
  lm.fit(X_train,y_train)

  coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])
  pd.set_option('display.max_rows', None)
  print(coeff_df.sort_values('Coefficient', ascending=False))

  predictions = lm.predict(X_test)

  ytest=y_test.values

  fig = px.scatter(x=ytest,y=predictions,width=1000, height=1000)
  fig.update_layout(yaxis_range=[-100,400])
  fig.update_layout(xaxis_range=[0,400])
  fig.show()

  print("min of y_test =",min(y_test),"max of y_test =", max(y_test))
  print("min of predictions =",min(predictions),"max of predictions =", max(predictions))

  print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [36]:
abnbSL.drop(['Cooking','Heating','Fridge','Wifi','Essentials','Air conditioning','Hot tub'],axis=1, inplace=True)

In [37]:
abnbSL.head()

,index,host_is_superhost,room_type,accommodates,bedrooms,price,review_scores_rating,new_bathroom,borough_Bronx,borough_Manhattan,borough_Queens,TV,Kitchen,Washer
0,0,0,4,1.00,0.00,225.00,4.70,1.00,0,1,0,1,1,0
1,1,0,2,2.00,1.00,60.00,4.52,1.00,0,0,0,0,1,0
2,3,0,2,2.00,1.00,68.00,4.23,1.00,0,1,0,1,0,0
3,4,0,2,1.00,1.00,75.00,4.91,1.00,0,1,0,0,0,0
4,5,1,4,2.00,1.00,135.00,4.68,1.00,0,1,0,0,1,0


In [38]:
LELR_all(abnbSL)

                      Coefficient
borough_Manhattan           36.85
room_type                   27.86
bedrooms                    17.21
TV                          14.96
Washer                      13.66
review_scores_rating        13.07
host_is_superhost           11.85
accommodates                11.10
new_bathroom                 7.87
borough_Queens             -15.03
borough_Bronx              -25.01
Kitchen                    -28.42


min of y_test = 10.0 max of y_test = 380.0
min of predictions = -3.8530273862439373 max of predictions = 387.18004630714034
RMSE: 57.68796860275919


In [39]:
  X = abnbSL.drop(['index','price'],axis=1)
  y = abnbSL['price']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=90)

  model = ltb.LGBMRegressor()
  model.fit(X_train, y_train)

  predictions = model.predict(X_test)
  ytest=y_test.values
  print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001004 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 177
[LightGBM] [Info] Number of data points in the train set: 12510, number of used features: 12
[LightGBM] [Info] Start training from score 138.956675
RMSE: 54.3916406378004


In [40]:
##model.booster_.save_model('lgbm_model.json')

In [41]:
import pickle

In [42]:
file = open('lgbm_model.pkl', 'wb')
pickle.dump(model, file)